# Matrix Analysis 2022 - EE312
## Week 9 - Eigenvalues
[LTS2](https://lts2.epfl.ch)

## Important
You need to submit *individually* your answers on moodle before the next exercise session (i.e. Monday or Friday depending if you are BA4/BA6). Answers to theoretical questions can be either written in the notebook or submitted separately.


## 1. RLC circuit
Let us consider a serial RLC circuit with a voltage source delivering a tension $v(t)$. A current $i(t)$ goes through the circuit. We denote by $v_R(t)$, $v_L(t)$, $v_C(t)$ the voltages in the circuit.

![RLC.png](../exercises/images/RLC.png)

Reminder: 
- $v_L(t) = L\frac{di(t)}{dt}$
- $\frac{dv_C(t)}{dt} = \frac{1}{C}i(t)$

**1.1** Using Ohm and Kirchoff laws, write the matrix first order differential equation describing the system. You can use $X(t) = \begin{pmatrix}v_C(t)\\i(t)\end{pmatrix}$.

$\frac{d}{dt}X(t) = A X(t) + b(t)$

with $v(t) = v_R(t) + v_L(t) + v_C(t) = R i(t) + L\frac{di(t)}{dt} + v_C(t)$ 

$ \Rightarrow A = \begin{pmatrix}
0 & \frac{1}{C}\\
-\frac{1}{L} & -\frac{R}{L} \\
\end{pmatrix} \qquad \text{ and } \qquad b(t) = \begin{pmatrix}0\\ \frac{v(t)}{L} \end{pmatrix}$

**1.2** What are the eigenvalues of the matrix in the equation you obtained ?

$\lambda_{1,2}  = \frac{1}{2L}\left(-R \pm \sqrt{R^2 - 4 \frac{L}{C}} \right)$

**1.3** Assuming $v(t) = 0$ and $X(0) = X_0$, compute $X(t)$

we have $x(t) = e^{(t-t_0)A}x_0$ with the spectral theorem we have $e^A = P e^{D} P^{-1}$ where $A = PDP^{-1}$ and $D$ is diagonal.

$\Rightarrow x(t) = P\begin{pmatrix} 
e^{(t-t_0)\lambda_1} & 0\\
0 & e^{(t-t_0)\lambda_2}\\
\end{pmatrix}
P^{-1}
$

**1.4** Let us use fixed $L$ and $C$ values. For which values of $R$ do we have real or complex eigenvalues ? Compute the limit value of $R$ for $L=40mH$ and $C=10nF$.

real: $\Delta = b^2 - 4ac = R^2 - 4\frac{L}{C} \geq 0 \Leftrightarrow R >= 2\sqrt{\frac{L}{C}}$

$R_{lim} = 2 \sqrt{\frac{40mH}{10nF}} = 4 k\Omega$ 

**1.5** Using $X_0=\begin{pmatrix}1.5\\ 10^{-3}\end{pmatrix}$ as initial conditions and (at least) two carefully chosen values of $R$, compute the solutions $v_C(t)$ and $i(t)$ and plot them. Be careful when computing and plotting the solutions, make sure you use the appropriate time range ! (hint: checking the value of the imaginary part of the eigenvalues might help). Discuss the different cases (do not forget the case $R=R_L$). You might use the [expm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm.html) function from scipy to compute a matrix exponential.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm


In [ ]:
def matrix(R, L, C):
    return np.array([[0, 1/C],[-1/L, -R/L]])

def plot_time(t, A, X_0):
    res = np.array([
        expm(_t*A)@X_0
        for _t in t 
    ])
    fig,ax = plt.subplots()
    ax.plot(t, res[:,0]) 
    ax.set_ylabel("voltage [V]")
    ax.set_xlabel("time")
    ax2 = ax.twinx()
    ax2.plot(t, res[:,1], color="red")
    ax2.set_ylabel("current [I]")
    fig.legend(["V","I"])
    plt.show()

def time(T):
    return np.linspace(0, T, 1000)

In [ ]:
x_0 = np.array([1.5, 1e-3])

L = 4e-2
C = 1e-8
R1 = 400
R_Lim = 4000
R3 = 40000

plot_time(time(1e-3), matrix(R1, L, C), x_0)
plot_time(time(1e-4), matrix(R_Lim, L, C), x_0)
plot_time(time(1e-5), matrix(R3, L, C), x_0)

## 2. Dynamics of opinion diffusion

Consider the dynamics of opinion diffusion among $N$ people sitting in a **ring-shaped structure**. Each individual is connected to her two nearest neighbors (i.e., left and right). Initially they have random opinions (represented as random real numbers), but at every time step, each individual changes his opinion depending on the social neighborhood. If we denote by $x_p(k)$ the value of the $p$-th individual at time step $k$, we will consider the following update rule:

$x_p(k+1) = (1 - 2\alpha)x_p(k) + \alpha(x_{p-1}(k) + x_{p+1}(k))$, $\alpha$ being a real-valued parameter.

Let us denote by $X(k) = \begin{pmatrix}x_1(k)\\x_2(k)\\ \vdots \\ x_N(k)\end{pmatrix}$.

**2.1** Write $X(k+1)$ as a function of $X(k)$ and as a function of X(0).

$X(k+1) = A X(k) = A^{k+1} X(0)$ with $A=\begin{pmatrix}1-2\alpha & \alpha & & \alpha\\ \alpha & 1-2\alpha & \ddots & \\ &  \ddots & \ddots & \alpha\\ \alpha&&\alpha & 1-2\alpha\end{pmatrix}$

**2.2** Using a vector containing $N$ random values for $X(0)$ implement a function that computes $X(k)$. Make sure you use as few matrix multiplications as possible.

In [ ]:
def update_matrix(N, alpha):
    return np.array(
        [
            [
                1 - 2*alpha if i == j else alpha if np.abs(i - j)==1 or np.abs(i - j)==N-1  else 0
                for j in range(N)
            ]
            for i in range(N)
        ]
    )

def compute_opinions(x0, update_matrix, num_steps):
    return np.linalg.matrix_power(update_matrix, num_steps)@x0

In [ ]:
print( update_matrix(6, 0.25))
compute_opinions(0.5*np.ones(6) + 0.2*np.random.rand(6), update_matrix(6, 0.25), 50)

**2.3** What are the eigenvalues of the matrix of the system ?

Hint: This result about circulant matrices might be helpful:

A $N\times N$ circulant matrix $C$ defined by:

$C = \begin{pmatrix}
c_1 & c_N & ... & c_3 & c_2\\
c_2 & c_1 & c_N & ... & c_3\\
\vdots & \vdots & ... & & \vdots\\
c_N & c_{N-1} & ... & c_2 & c_1
\end{pmatrix}$

has its eigenvalues defined by:

$\lambda_k = c_1 + c_N\omega^k + c_{N-1}\omega^{2k} + ... + c_2\omega^{k(N-1)}, k=0, 1, ..., N-1$ and $\omega=e^{\frac{2i\pi}{N}}$. 

**Answer**
We have $c_1 = 1-2\alpha$, $c_2 = c_N = \alpha$ and $c_3 = c_4 = ... = c_{N-1} = 0$ 

Therefore we have $ \lambda_k = 1-2\alpha + \alpha \omega^k +  \alpha \omega^{k(N-1)}  $ and $\omega=e^{\frac{2i\pi}{N}}$

$ \lambda_k =  1 - 2\alpha + \alpha e^{\frac{2i\pi k}{N}} +  \alpha e^{\frac{2i\pi k(N-1)}{N}}  =  1 - 2\alpha + \alpha e^{\frac{2i\pi k}{N}} +  \alpha e^{\frac{-2i\pi k}{N}} = 1 - 2\alpha + \alpha e^{\frac{2i\pi k}{N}} +  \alpha e^{\frac{-2i\pi k}{N}} = 1 - 2\alpha + 2 \alpha cos{\frac{2\pi k}{N}} $

**2.4** Implement a function that returns all opinion values, i.e. a vector $\begin{pmatrix}X(0)\\ X(1)\\ X(2)\\ \vdots\\ X(p)\end{pmatrix}$. Plot the evolution of the opinion values. Choose a small value of $N$ (e.g. 5 or 6) and consider $\alpha\in[-\frac{1}{2}, \frac{1}{2}]$. How many different behaviors can you identify ? How do they relate to the eigenvalues of the update matrix ? 

**Answer** 

1. $ \alpha = 0 \quad \Rightarrow \lambda_k = 1  \qquad\forall k$ (constant)
2. $ \alpha \in \left]0, 1/2\right[ \quad \Rightarrow  \lambda_k = 1 - 2\alpha + 2 \alpha cos{\frac{2\pi k}{N}}  \Rightarrow -1 < \lambda_k \leq 1 $  (converge)
3. $ \alpha \in \left[-1/2, 0\right[ \quad \Rightarrow \lambda_k = 1 + 2 |\alpha| - 2 |\alpha| cos{\frac{2\pi k}{N}}  \Rightarrow 1 \leq \lambda_k \leq 2$ (diverge)
4. $ \alpha =  1/2 \quad\Rightarrow \lambda_k = cos{\frac{2\pi k}{N}} \Rightarrow \lambda_N = 1 \quad \text{if N even} \quad \lambda_{N/2} = -1 $ (oscilation) 
 

In [ ]:
def compute_opinions_evol(x0, update_matrix, num_steps):
    return [
        compute_opinions(x0, update_matrix, step)
        for step in range(num_steps)
    ]

In [ ]:
num_steps = 50


def test_for_N_alpha(N, alpha):
    x0 = 0.5*np.ones(N) + 0.2*np.random.rand(N)

    A = update_matrix(N, alpha)
    plt.plot(range(num_steps), compute_opinions_evol(x0, A, num_steps))
    plt.show()





In [ ]:
N = 5 
test_for_N_alpha(N, -0.1)    # diverges 
test_for_N_alpha(N, 0)       # staionnairy
test_for_N_alpha(N, 0.2)     # converges
test_for_N_alpha(N, 0.4)     # converges
test_for_N_alpha(N, 0.5)     # converges "oscilating" for N ODD
test_for_N_alpha(N, 0.55)    # converges slowly "oscilating"


In [ ]:
N = 6
test_for_N_alpha(N, -0.1)    # diverges 
test_for_N_alpha(N, 0)       # staionnairy
test_for_N_alpha(N, 0.2)     # converges
test_for_N_alpha(N, 0.4)     # converges
test_for_N_alpha(N, 0.5)     # oscilates for N EVEN
test_for_N_alpha(N, 0.55)    # diverges "oscilating" for N EVEN